<br>

# Maps

In [ ]:
#!pip3 install open-geodata --upgrade

In [ ]:
import os
import json
import fiona
import folium
import pandas as pd
import branca as bc
import seaborn as sns
import geopandas as gpd
from datetime import date
from folium import plugins

In [ ]:
from open_geodata import geo, lyr

In [ ]:
from paths import *

<br>

# Layers

<br>

## Municípios

In [ ]:
def limite_mun():    
    # Input
    gdf = gpd.read_file(os.path.join(output_path_gpkg, 'municipios_gaema.gpkg'))
    gdf = gdf.to_crs(epsg=4326)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Municípios',
        style_function=lambda x: {
            'fillColor': '#FFD700',
            'color': '#FFD700',
            'weight': 1,
            'fillOpacity': 0.0
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', ],
            aliases=['Município', ],
            labels=True,
            sticky=True,
            opacity=0.9,
            direction='auto',
        ),        
        highlight_function=lambda x: {
            'weight': 3
        },
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )

In [ ]:
def style_function(feature):
    default_style = {
        #'opacity': 0.0,
        'fillColor': '#bfaf00',
        'fillOpacity' : 0.5,
        'color': 'black',
        'weight': 1
    }
        
    if feature['properties']['Restrito']  == 'Sim':
        default_style['fillColor'] = '#f43b13'
        
    if feature['properties']['Restrito']  == 'Não':
        default_style['fillColor'] = '#15bd3b'
                
    return default_style


def restricoes():
    # Input
    gdf = gpd.read_file(os.path.join(output_path_gpkg, 'restricao_merged_mun.gpkg'))
    gdf = gdf.to_crs(epsg=4326)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Restrições',
        style_function=style_function,
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )

<br>

# Maps

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]

    # Zoom
    min_zoom = 10
    max_zoom = 18

    # Create Map
    m = folium.Map(
        #location=list_centroid,
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )

    # Add Base Layers
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))

    
    m.add_child(restricoes())
    m.add_child(limite_mun())

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
# ddd    
m = get_map(os.path.join(output_path_gpkg, 'municipios_gaema.gpkg'))

# Save/Open Map
map_file = os.path.join(output_path_maps, 'map_ic-16-16.html')
m.save(map_file)
m